In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Load the Boston Housing dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.boston_housing.load_data()

In [6]:
# Normalize the dataset
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Split a validation set from the training data
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Define a basic ANN model
def build_ann_model():
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)  # No activation function for regression output
    ])

    model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['mae'])
    return model

# Build and train the model
ann_model = build_ann_model()
history = ann_model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val), batch_size=32)

# Evaluate the model on the test data
test_loss, test_mae_without_tuner = ann_model.evaluate(x_test, y_test)
print(f"Test MAE without Keras Tuner: {test_mae_without_tuner:.4f}")

# Calculate R² for the model without Keras Tuner
y_pred_without_tuner = ann_model.predict(x_test)
r2_without_tuner = r2_score(y_test, y_pred_without_tuner)
print(f"R² without Keras Tuner: {r2_without_tuner:.4f}")

C:\Users\Skuad\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 600.4018 - mae: 22.7224 - val_loss: 431.4443 - val_mae: 19.4359
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 547.7275 - mae: 21.3500 - val_loss: 389.7696 - val_mae: 18.2200
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 486.8806 - mae: 19.7835 - val_loss: 340.2635 - val_mae: 16.6798
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 435.9070 - mae: 18.3976 - val_loss: 279.2024 - val_mae: 14.8239
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 328.4661 - mae: 15.5236 - val_loss: 211.7094 - val_mae: 12.4871
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 251.1805 - mae: 13.0827 - val_loss: 145.4991 - val_mae: 9.9771
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 171.6889 - mae: 10.5767 - val_loss: 96.0279 - val_mae: 7.9294
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 112.4082 - mae: 8.1501 - val_loss: 69.4321 - val_mae: 6.6018
Epoch 9/10
11/11 ━━━━━

In [7]:
import keras_tuner as kt

In [8]:
from keras_tuner import RandomSearch

# Define a model-building function for Keras Tuner
def build_ann_model_with_tuner(hp):
    model = keras.Sequential()

    # Tune the number of layers
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
                               activation='relu'))

    # Output layer for regression
    model.add(layers.Dense(1))

    # Tune the optimizer type
    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop'])
    model.compile(optimizer=optimizer,
                  loss='mse',
                  metrics=['mae'])

    return model

# Set up the tuner
tuner = RandomSearch(
    build_ann_model_with_tuner,
    objective='val_mae',
    max_trials=10,
    executions_per_trial=2,
    directory='tuner_dir',
    project_name='boston_housing_ann_tuning'
)

# Search for the best hyperparameters
tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val), batch_size=32)

# Get the best model
best_ann_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model on the test data
best_test_loss, best_test_mae_with_tuner = best_ann_model.evaluate(x_test, y_test)
print(f"Test MAE with Keras Tuner: {best_test_mae_with_tuner:.4f}")

# Calculate R² for the model with Keras Tuner
y_pred_with_tuner = best_ann_model.predict(x_test)
r2_with_tuner = r2_score(y_test, y_pred_with_tuner)
print(f"R² with Keras Tuner: {r2_with_tuner:.4f}")

Trial 10 Complete [00h 00m 12s]
val_mae: 2.8162994384765625

Best val_mae So Far: 2.576998233795166
Total elapsed time: 00h 01m 42s


C:\Users\Skuad\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 8 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 18.2227 - mae: 3.2208 
Test MAE with Keras Tuner: 3.3547
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
R² with Keras Tuner: 0.7338


In [9]:
# Compare MAE
print(f"Test MAE without Keras Tuner: {test_mae_without_tuner:.4f}")
print(f"Test MAE with Keras Tuner: {best_test_mae_with_tuner:.4f}")

# Compare R²
print(f"R² without Keras Tuner: {r2_without_tuner:.4f}")
print(f"R² with Keras Tuner: {r2_with_tuner:.4f}")

if best_test_mae_with_tuner < test_mae_without_tuner:
    print("The model with Keras Tuner has a better MAE.")
else:
    print("The model without Keras Tuner has a better MAE.")

if r2_with_tuner > r2_without_tuner:
    print("The model with Keras Tuner has a better R².")
else:
    print("The model without Keras Tuner has a better R².")

Test MAE without Keras Tuner: 5.9122
Test MAE with Keras Tuner: 3.3547
R² without Keras Tuner: 0.2683
R² with Keras Tuner: 0.7338
The model with Keras Tuner has a better MAE.
The model with Keras Tuner has a better R².
